### Importando Bibliotecas:

In [9]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Criando Funções:

In [10]:
def converter_df(aluguel_lista, quartos_lista, banheiro_lista, vaga_lista, area_lista):
    data = {'aluguel': aluguel_lista,
           'quartos': quartos_lista,
           'banheiros':banheiro_lista,
           'vagas':vaga_lista,
           'area':area_lista}
    df = pd.DataFrame(data)
    return df

In [11]:
def scrap(url):
    c = requests.get(url).content
    soup = BeautifulSoup(c)
    
    aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'})
    aluguel_lista = [p.text for p in aluguel_tag]
    
    quartos_tag = soup.find_all('li', {'class':'property-card__detail-item property-card__detail-room js-property-detail-rooms'}) 
    quartos_lista = [p.text for p in quartos_tag]
    
    banheiro_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'}) 
    banheiro_lista = [p.text for p in banheiro_tag]
    
    vaga_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-garage js-property-detail-garages'}) 
    vaga_lista = [p.text for p in vaga_tag]
    
    area_tag = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-area'}) 
    area_lista = [p.text for p in area_tag]
    
    df = converter_df(aluguel_lista, quartos_lista, banheiro_lista, vaga_lista, area_lista)
    return df

In [12]:
paginas = ['']
for i in range(2,10):
    a = '?pagina='+str(i)
    paginas.append(a)

In [13]:
URL = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/{}'

### Web Scraping com BeautifulSoup:

In [14]:
df_lista = []
for pg in paginas:
    url = URL.format(pg)
    print('Coletando dados da URL:', url)
    try:
        df = scrap(url)
    except Exception as e:
        print('Erro ao coletar dados da url:', url)
        print(e)
    df_lista.append(df)
    #time.sleep(30)
df_final = pd.concat(df_lista)

Coletando dados da URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/
Coletando dados da URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=2
Coletando dados da URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=3
Coletando dados da URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=4
Coletando dados da URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=5
Coletando dados da URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=6
Coletando dados da URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=7
Coletando dados da URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=8
Coletando dados da URL: https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina=9


In [15]:
df_final.head()

,aluguel,quartos,banheiros,vagas,area
0,R$ 2.200 /Mês,1 Quarto,1 Banheiro,1 Vaga,37 m²
1,R$ 2.900 /Mês,1 Quarto,2 Banheiros,-- Vaga,43 m²
2,R$ 2.200 /Mês,1 Quarto,1 Banheiro,-- Vaga,33 m²
3,R$ 1.700 /Mês,2 Quartos,2 Banheiros,1 Vaga,57 m²
4,R$ 2.100 /Mês,2 Quartos,1 Banheiro,1 Vaga,50 m²


In [16]:
df_final.shape

(324, 5)

In [17]:
df_final2 = df_final.drop_duplicates(subset=['aluguel','quartos','banheiros','vagas','area'], keep='first')

In [18]:
df_final2.shape

(36, 5)

In [19]:
df_final.to_csv("dados_brutos.csv")